# Introduction to Prometheus


[Prometheus](https://prometheus.io/) is a metric oriented monitoring system. This brief introduction is to evaluate its use for LSST DM in implementing some aspects of the SQuaSH monitoring and the DM Engineering Facilities Database (DM-EFD) monitoring.

## Concepts

Prometheus is a pull-based monitoring system, which means that a Prometheus client exposes metrics via an HTTP endpoint, and the Prometheus server scrapes them periodically.

Prometheus support different [types of metrics](https://prometheus.io/docs/concepts/metric_types/). A "metric value" in Prometheus is called a sample. A single Prometheus server can ingest millions of samples per second. The result is a large number of stored time series, which are uniquely identified by a [metric name](https://prometheus.io/docs/practices/naming/) and a set of labels. The Prometheus project implements its own Time Series DB and query language (PromQL) in Go.

The PromQL query language supports several [operators](https://prometheus.io/docs/prometheus/latest/querying/operators/#comparison-binary-operators), different types of [aggregations](https://prometheus.io/docs/prometheus/latest/querying/operators/#aggregation-operators) and [functions](https://prometheus.io/docs/prometheus/latest/querying/functions/) that can be applied to the time series.

Alerting with Prometheus requires two steps, one is creating [alerting rules](https://prometheus.io/docs/prometheus/latest/configuration/alerting_rules/) in PromQL and then configuring the [alert manager](https://prometheus.io/docs/alerting/configuration/). 
 

## Starting a Prometheus server 

The next cell will start a Prometheus server at http://localhost:9090 using the `prometheus.yml` example configuration file in the repository. 

In [ ]:
%%bash --bg
docker run --rm -p 9090:9090 -v ${PWD}/prometheus.yml:/etc/prometheus/prometheus.yml --name prometheus prom/prometheus

Note: you can stop the server by interrupting the Jupyter notebook kernel.

In [ ]:
!cat prometheus.yml

The Prometheus application itself is instrumented with Prometheus metrics. The above configuration tells the Prometheus server to scrape the Prometheus metrics every 10s. In this case the target is the Prometheus server and the metrics are exposed by default at http://localhost:9090/metrics.

After starting the server, you might type `prometheus_http_response_size_bytes_count` in the [Prometheus expression browser](http://localhost:9090/graph). This will return all time series that have `prometheus_http_response_size_bytes_count` as the metric name.

The PromQL query language allows filtering and aggregation based on labels. A query like
`prometheus_http_response_size_bytes_count{handler="/graph"}` will show results only for the `/graph` endpoint.

Another interesting query `rate(prometheus_tsdb_head_samples_appended_total[1m])` uses the `rate` function on the `prometheus_db_head_samples_appended_total` metric and returns the Prometheus server ingest rate averaged over one minute.

## Example of a Prometheus client in Python 


The next cell runs the `demo.py` app in background.

In [ ]:
%%bash --bg
python demo.py

In [ ]:
!cat demo.py

The demo app creates a `demo_request_processing_time` metric that can be used to monitor the total time spent in processing requests. The "request processing" is implemented as  `time.sleep(random.random())` for illustration purposes.

The Prometheus server can now scrape the new metric exposed by the demo app at http://localhost:8000/metrics. In order to do that, you have to add a new target to the server configuration.

In [ ]:
!cat client-template.yml

where `{{ DOCKER_HOST_IP }}` is the IP address of the Docker host. It can be obtained, for example, with `ip addr show docker0` and it is needed here so that the Prometheus server running inside the Docker container can connect to the demo client running on the local host.

After creating the `client.yml` file with the new configuration, the next cell will restart the Prometheus server.

In [ ]:
%%bash --bg
docker stop prometheus
docker run --rm -p 9090:9090 -v ${PWD}/client.yml:/etc/prometheus/prometheus.yml --name prometheus prom/prometheus

You should see the new metric `demo_request_processing_time` from the Prometheus expression browser at http://localhost:9090/graph.

## Visualization


[Console templates](https://prometheus.io/docs/visualization/consoles/) can be used to extend the Prometheus server UI. Another common solution is the integration with [Graphana](https://prometheus.io/docs/visualization/grafana/).

## Alerting with Prometheus

[Alerting rules](https://prometheus.io/docs/prometheus/latest/configuration/alerting_rules/) are usually configured in a separate file. For example, the `rules.yml` file specifies an hypotetical alerting rule for high request processing times. In this case, the alert is generated if the expression is evaluated to `True` for 1 minute.

In [ ]:
!cat rules.yml

The evaluation interval and the `rules.yml` file itself must be added to the Prometheus server configuration.

In [ ]:
!cat alerting-template.yml

Note: for simplicity the alert manager configuration is omitted here.
    
The next cell restarts the Prometheus server, assuming you have updated the `alerting.yml` configuration file.

In [ ]:
%%bash --bg
docker stop prometheus
docker run --rm -p 9090:9090 -v ${PWD}/alerting.yml:/etc/prometheus/prometheus.yml -v ${PWD}/rules.yml:/etc/prometheus/rules.yml --name prometheus prom/prometheus

Finally, the alerting rules can be seen from the Prometheus server UI at http://localhost:9090/alerts.

## Discussion

Prometheus is largely used for infrastructure and application monitoring. The main reason we don't recommend it for LSST DM monitoring is because it is a pull-based monitoring system. Even if there are options like [PushGateway](https://github.com/prometheus/pushgateway) for pushing metrics from short lived jobs to   an intermediary cahe which Prometheus can scrape, the time series resolution is fixed by the `scrape_interval` configuration. That doesn't fit well for SQuaSH and for the DM-EFD use cases where we want to collect, store and visualize eventually all "metric measurements".

## References
- https://prometheus.io/

- [Prometheus Up and Running](https://www.oreilly.com/library/view/prometheus-up/9781492034131/)

